In [ ]:
import sys
sys.path.append('..')
from scripts.data_analysis import DataAnalysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
original_df = pd.read_csv('../data/data_wave1.csv')

In [ ]:
columns_to_extract = [
    'aApoB', 'aHDL_C', 'aTotFA', 'aSerum_TG', 'aGp', 'aIle', 'Sexe', 'Age',
    'aauditsc', 'aedu', 'AIPMETO2', 'asmokstat', 'acidep09', 'anumdis_treat', 'abaiscal', 'ahsCRP', 'aIL6',
    'aIRSsum9', 'ams_waist', 'ams_hpt', 'ams_trig2', 'ams_hdl2', 'ams_gluc2',
    'amet_syn2', 'atri_med', 'ahdl_med', 'asbp_med', 'adbp_med', 'agluc_med'
]

extracted_df = original_df[columns_to_extract]

extracted_csv_file = '../data/wave1_data_to_discretize.csv'
extracted_df.to_csv(extracted_csv_file, index=False)

In [ ]:
def classify_data(df):
    """
    Classify each column in a pandas DataFrame as continuous, discrete, binary, or categorical.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to classify.

    Returns:
    dict: A dictionary with column names as keys and their classifications as values.
    """
    def classify_series(series):
        """
        Classify a pandas Series as continuous, discrete, binary, or categorical.
        
        Parameters:
        series (pd.Series): The Series from a column to classify.

        Returns:
        str: The classification of the series ('continuous', 'discrete', 'binary', 'categorical').
        """
        if pd.api.types.is_numeric_dtype(series):
            unique_count = series.nunique()
            total_count = len(series)
            
            if unique_count == 2:
                return 'binary'
            elif unique_count < 10:
                return 'discrete'
            else:
                return 'continuous'
        else:
            return 'categorical'

    column_classifications = {col: classify_series(df[col]) for col in df.columns}
    return column_classifications

In [ ]:
data = pd.read_csv('../data/wave1_data_to_discretize.csv')

In [ ]:
# replace -1 in columns with NaN
columns_with_minus_one = [
    'ahsCRP', 'aIL6', 'ams_waist', 'ams_hpt', 'ams_trig2', 'ams_hdl2', 'ams_gluc2',
    'amet_syn2'
]

for col in columns_with_minus_one:  
  data[col] = data[col].replace(-1, np.nan)

# in sleeping pattern, -3 and -2 means no data
data['aIRSsum9'] = data[col].replace(-3, np.nan)
data['aIRSsum9'] = data[col].replace(-2, np.nan)

In [ ]:
column_types = classify_data(data)
column_types

In [ ]:
DataAnalysis.plot_missing_values(data)

In [ ]:
from scipy.stats import chi2_contingency

def perform_little_mcar_test(df):
    results = []

    for col_with_missing_data in df.columns[df.isnull().any()]:
        # Create a mask indicating where data is missing
        missing_data = df[col_with_missing_data ].isnull().astype(int)

        # Create contingency table for Chi-Squared test
        contingency_table = pd.crosstab(missing_data, df.drop(columns=[col_with_missing_data]).isnull().any(axis=1).astype(int))

        # Perform Chi-Squared test
        chi2_stat, p_val, dof, _ = chi2_contingency(contingency_table, correction=False)

        # Append results
        results.append({'Column': col_with_missing_data, 'Chi-Squared': chi2_stat, 'df': dof, 'p-value': p_val})

    return pd.DataFrame(results)

perform_little_mcar_test(data)

Data is not missing completely at random so we cannot discard it

In [ ]:
DataAnalysis.plot_correlation_matrix(data, columns_to_extract, 'Variables to Construct Bayesian Network')

Now, we are going to determine the imputation strategy for each column with missing data based on correlation matrix

In [ ]:
corr_matrix = data.corr().abs()


In [ ]:
from sklearn.impute import KNNImputer
from sklearn.impute import IterativeImputer

def impute_missing_values(df, corr_matrix, column_types):
    imputed_df = df.copy()

    if 'amet_syn2' in df.columns:
        correlated_vars = ['ams_trig2', 'ams_hdl2', 'ams_hpt', 'ams_gluc2', 'atri_med', 'ahdl_med', 'asbp_med', 'adbp_med', 'agluc_med']
        imputation_vars = ['amet_syn2'] + correlated_vars
        data_for_imputation = df[imputation_vars]
        
        iter_imputer = IterativeImputer(max_iter=100, random_state=42)
        imputed_data = iter_imputer.fit_transform(data_for_imputation)
        # convert to binary
        imputed_df['amet_syn2'] = np.round(imputed_data[:, 0]).astype(int)

    for col in df.columns[df.isnull().any()]:
        if col == 'amet_syn2':
            continue

        col_corr = corr_matrix[col].drop(col)
        max_corr = col_corr.max()

        # use MICE for highly correlated columns
        if max_corr > 0.6:
            mice_imputer = IterativeImputer(random_state=42)
            imputed_df[col] = mice_imputer.fit_transform(df[[col] + col_corr.index.tolist()])[:, 0]
        # use kNN for everything else
        else:
            knn_imputer = KNNImputer(n_neighbors=10)
            imputed_df[col] = knn_imputer.fit_transform(df[[col] + col_corr.index.tolist()])[:, 0]
        
        if column_types[col] == 'binary':
            imputed_df[col] = np.round(imputed_df[col]).astype(int)

    return imputed_df

In [ ]:
imputed_df = impute_missing_values(data, corr_matrix, column_types)

In [ ]:
DataAnalysis.plot_missing_values(imputed_df)

In [ ]:
classify_data(imputed_df)

In [ ]:
imputed_df.to_csv('../data/network/imputed_data_wave1.csv')

## Transform continuous variables to discrete

In [ ]:
preprocessed_df = pd.read_csv('../data/network/imputed_data_wave1.csv')

In [ ]:
from sklearn.cluster import KMeans
def apply_kmeans(df, column_name, n_clusters=3, random_state=0):
    data = df[column_name].values.reshape(-1, 1)
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state).fit(data)
    
    # get centroids and create sorted labels
    centroids = kmeans.cluster_centers_
    sorted_idx = np.argsort(centroids.ravel())
    labels_sorted = ['Low', 'Moderate', 'High']
    
    # map original cluster labels to meaningful labels
    cluster_mapping = {old_label: new_label for old_label, new_label in zip(sorted_idx, labels_sorted)}
    df[column_name] = kmeans.labels_
    df[column_name] = df[column_name].map(cluster_mapping)
    
    # visualize
    colors = ['#3B28CC', '#3F8EFC', '#ADD7F6']
    plt.figure(figsize=(8, 6))
    for label in labels_sorted:
        subset = df[df[column_name] == label]
        plt.hist(subset[column_name], bins=np.arange(n_clusters + 1) - 0.5, color=colors[labels_sorted.index(label)], label=label, alpha=0.75, edgecolor='black')

    plt.title(f'Clusters of {column_name}')
    plt.xlabel('Label')
    plt.ylabel('Frequency')
    plt.xticks(range(n_clusters), labels_sorted) 
    plt.legend()
    plt.show()
    
    return df


In [ ]:
#aApoB
bins = [-float('inf'), 1.3, float('inf')]
labels = ['Low', 'High']
preprocessed_df['aApoB'] = pd.cut(preprocessed_df['aApoB'], bins=bins, labels=labels, right=False)
preprocessed_df['aApoB'].unique()


In [ ]:
#aTotFA
preprocessed_df = apply_kmeans(preprocessed_df, 'aTotFA', n_clusters=3, random_state=0)


In [ ]:
preprocessed_df['aHDL_C'] = preprocessed_df.apply(
    lambda row: 'Desirable' if (row['Sexe'] == 1 and row['aHDL_C'] > 1.03) or (row['Sexe'] == 2 and row['aHDL_C'] > 1.29) else 'At risk',
    axis=1
)

In [ ]:
bins = [-float('inf'), 1.69, 2.26, 5.65, float('inf')]
labels = ['Desirable', 'Mild Hypertriglyceridemia', 'High Hypertriglyceridemia', 'Very High Hypertriglyceridemia']
preprocessed_df['aSerum_TG'] = pd.cut(preprocessed_df['aSerum_TG'], bins=bins, labels=labels)
preprocessed_df['aSerum_TG'].unique()


In [ ]:
preprocessed_df['aGp'] = preprocessed_df['aGp'].apply(lambda x: 'Normal' if x <= 1.2 else 'At risk')
preprocessed_df['aGp'].unique()

In [ ]:
preprocessed_df = apply_kmeans(preprocessed_df, 'aIle', n_clusters=3, random_state=0)



In [ ]:
def categorize_met(met):
    moderate = 4.9 * 60 * 3 # assuming 3 sessions of 1 hour each per week
    vigorous = 6.8 * 60 * 3
    if met >= moderate and met < vigorous:  
        return 'Moderate'
    elif met >= 6.8 * 60 * 3: 
        return 'Vigorous'
    else:
        return 'Low'

preprocessed_df['AIPMETO2'] = preprocessed_df['AIPMETO2'].apply(categorize_met)
print(preprocessed_df['AIPMETO2'].value_counts())

In [ ]:
preprocessed_df['aIL6'] = pd.cut(preprocessed_df['aIL6'],
                                bins=[-float('inf'), 7, float('inf')],
                                labels=['Normal', 'High'],
                                right=True) 

print(preprocessed_df['aIL6'].value_counts())

In [ ]:
preprocessed_df['ahsCRP'] = pd.cut(preprocessed_df['ahsCRP'],
                                  bins=[-float('inf'), 0.3, 1, 10, 50, float('inf')],
                                  labels=['Normal', 'Minor', 'Moderate', 'Marked', 'Severe'],
                                  right=False)
print(preprocessed_df['ahsCRP'].value_counts())

In [ ]:
def categorize_atri_med(row):
    adult_threshold = 150 * 0.01129  # mg/dL to mmol/L for adults
    child_threshold = 90 * 0.01129   # mg/dL to mmol/L for children and teens

    if row['Age'] > 19:
        if row['atri_med'] < adult_threshold:
            return 'Healthy'
        else:
            return 'High'
    else: 
        if row['atri_med'] < child_threshold:
            return 'Healthy'
        else:
            return 'High'

preprocessed_df['atri_med'] = preprocessed_df.apply(categorize_atri_med, axis=1)
print(preprocessed_df['atri_med'].value_counts())


In [ ]:
def categorize_ahdl_med(row):
    if row['Sexe'] == 1: 
        return 'Healthy' if row['ahdl_med'] > 1.0 else 'Unhealthy'
    elif row['Sexe'] == 2: 
        return 'Healthy' if row['ahdl_med'] > 1.2 else 'Unhealthy'

preprocessed_df['ahdl_med'] = preprocessed_df.apply(categorize_ahdl_med, axis=1)
preprocessed_df['ahdl_med'].value_counts()

In [ ]:
def categorize_blood_pressure(df):
    for index, row in df.iterrows():
        sbp = row['asbp_med']
        dbp = row['adbp_med']
        
        if sbp < 122 and dbp < 80:
            df.at[index, 'asbp_med'] = 'Normal'
            df.at[index, 'adbp_med'] = 'Normal'
        elif 120 <= sbp <= 129 and dbp < 80:
            df.at[index, 'asbp_med'] = 'Elevated'
            df.at[index, 'adbp_med'] = 'Elevated'
        elif (130 <= sbp <= 139 or 80 <= dbp <= 89):
            df.at[index, 'asbp_med'] = 'Hypertension I'
            df.at[index, 'adbp_med'] = 'Hypertension I'
        elif sbp >= 140 or dbp >= 90:
            df.at[index, 'asbp_med'] = 'Hypertension II'
            df.at[index, 'adbp_med'] = 'Hypertension II'
        else:
            df.at[index, 'asbp_med'] = 'Uncategorized'
            df.at[index, 'adbp_med'] = 'Uncategorized'

categorize_blood_pressure(preprocessed_df)

In [ ]:
def categorize_glucose_level(glucose_mmol):
    glucose_mg_dl = glucose_mmol * 18  # convert mmol/L to mg/dL
    if glucose_mg_dl < 70:
        return 'Hypoglycemia'
    elif 70 <= glucose_mg_dl <= 100:
        return 'Normal'
    elif 100 < glucose_mg_dl <= 125:
        return 'Prediabetes'
    elif glucose_mg_dl >= 126:
        return 'Diabetes'
    else:
        return 'Uncategorized' 

preprocessed_df['agluc_med'] = preprocessed_df['agluc_med'].apply(categorize_glucose_level)

preprocessed_df['agluc_med'].value_counts()

In [ ]:
# save data to build the network
preprocessed_df.to_csv('../data/network/final_data_wave1.csv', index=False)

In [ ]:
final_data = pd.read_csv('../data/network/final_data_wave1.csv')
classify_data(final_data)